### 랭체인 준비

In [1]:
# 패키지 설치
!pip install langchain==0.3.7
!pip install langchain-google-genai
!pip install langchain_community
!pip install langgraph

In [2]:
import os
from google.colab import userdata

# 환경 변수 준비(좌측 상단의 열쇠 아이콘으로 GOOGLE_API_KEY 설정)
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

###랭스미스

In [3]:
import os
from uuid import uuid4

# 환경 변수 준비
unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}"

### 임베딩 모델 준비

In [4]:
# 허깅 페이스 패키지 설치
!pip install langchain-huggingface

In [5]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

# 임베딩 모델 준비
hf_embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


### 도구 준비

#### 타빌리 도구 준비

In [51]:
import os
from google.colab import userdata

# 환경 변수 준비(좌측 상단의 열쇠 아이콘으로 TAVILY_API_KEY 설정)
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

In [59]:
from langchain_community.tools import TavilySearchResults

# 타빌리 도구 준비
tavily_tool = TavilySearchResults(max_results=5)

In [60]:
# 타빌리 도구 실행
tavily_tool.invoke("대한민국의 수도는 어디인가요?")

[{'url': 'https://www.a-ha.io/questions/40159372a1dc09e3b630d0a0543c615c',
  'content': '한국의 수도는 어디인가요? 알려주세요 - 한국 수도는 어디인가요?서울 대전 대구 부산 인천 평양 신의주 개성 제주 익산 서산 천안 세종 군산 남산 남포 경주 경산 김해 아산 예산어디지? ... 대한민국의 수도는 서울 입니다..서울엔 많은게 다 유치하고 있지요'},
 {'url': 'https://ko.wikipedia.org/wiki/수도',
  'content': '대한민국의 수도는 서울이다. 대만의 수도 타이베이. 수도(한국 한자: 首都, 영어: Capital city)는 한 국가의 정치, 행정의 중심이 되는 도시를 말한다. 수도에는 대부분 중앙정부가 소재해, 국가원수 등 국가의 최고 지도자가 거점으로 두는 도시이다. 다만 중앙 정부의 소재와는 별도로 그 나라의'},
 {'url': 'https://online1000book.com/entry/대한민국KOREA에-대하여',
  'content': '대한민국의 수도는 어디인가요? 대한민국의 수도는 서울입니다. 대한민국의 언어는 무엇인가요? 한국어(한글)를 사용하며, 공용어로 지정되어 있습니다. 영어는 교육 및 업무에서 자주 사용되기도 합니다. 대한민국의 인구는 얼마나 되나요?'},
 {'url': 'https://www.lawtimes.co.kr/Legal-Opinion/Legal-Opinion-View?serial=171373',
  'content': "신행정수도법은 헌법을 이렇게 개정하였다. '대한민국의 수도는 대전광역시·충청북도 및 충청남도 일원이다.'. 거칠되 간단하게 말하면 '대한민국의 수도는 세종시다.'. 헌재의 논리를 따른다면 현재 대한민국의 (불문)헌법상 수도는 세종시다. 4. 결론"},
 {'url': 'https://ko.wikipedia.org/wiki/한국의_수도',
  'content': '미군정에 의하여\xa0경기도에서 분리해

#### 검색 도구 준비

In [9]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 문서 준비
loader = WebBaseLoader("https://python.langchain.com/")
documents = loader.load()
print(len(documents))

1


In [10]:
# 문서 분할
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(documents)
print(len(documents))

18


In [11]:
# 랭체인 크로마 패키지 설치
!pip install langchain-chroma

In [12]:
from langchain_chroma import Chroma

# 벡터 스토어 준비
vectorstore = Chroma.from_documents(
    documents,
    embedding=hf_embeddings,
)

# 검색 도구 준비
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 2},
)

In [13]:
from langchain.tools.retriever import create_retriever_tool

# 검색 도구 준비
retriever_tool = create_retriever_tool(
    retriever,
    "langchain_search",
    "Langchain에 관한 정보를 검색합니다. Langchain에 관해 질문이 있다면, 이 도구를 사용해주세요.",
)

In [14]:
# 검색 도구 실행
retriever_tool.invoke("Langsmith에 관해 알려주세요.")

'Introduction | \uf8ffü¶úÔ∏è\uf8ffüîó LangChain\n\n(self-constructing) chainText embedding modelsHow to combine results from multiple retrieversHow to select examples from a LangSmith datasetHow to select examples by lengthHow to select examples by maximal marginal relevance (MMR)How to select examples by n-gram overlapHow to select examples by similarityHow to use reference examples when doing extractionHow to handle long text when doing extractionHow to use prompting alone (no tool calling) to do extractionHow to add fallbacks to a runnableHow to filter messagesHybrid SearchHow to use the LangChain indexing APIHow to inspect runnablesLangChain Expression Language CheatsheetHow to cache LLM responsesHow to track token usage for LLMsRun models locallyHow to get log probabilitiesHow to reorder retrieved results to mitigate the "lost in the middle" effectHow to split Markdown by HeadersHow to merge consecutive messages of the same typeHow to add message historyHow to migrate from legacy 

### 에이전트 구현

In [15]:
from langchain_google_genai import ChatGoogleGenerativeAI

# LLM 준비
llm = ChatGoogleGenerativeAI(
    model="models/gemini-1.5-flash",
)

In [54]:
from langgraph.prebuilt import chat_agent_executor

# 에이전트 준비
agent_executor = chat_agent_executor.create_tool_calling_executor(
    llm,
    tools=[tavily_tool, retriever_tool]
)

In [55]:
# 도구를 사용하지 않은 질의 응답
response = agent_executor.invoke(
    {"messages": [("human", "안녕하세요.")]}
)
response["messages"][-1].content

'안녕하세요! 무엇을 도와드릴까요?\n'

In [58]:
# 타빌리 도구를 사용한 질의 응답
response = agent_executor.invoke(
    {"messages": [("human", "대한민국의 수도는 어디인가요?")]}
)
response["messages"][-1].content

'대한민국의 수도는 서울입니다.\n'

In [19]:
# 검색 도구를 사용한 질의 응답
response = agent_executor.invoke(
    {"messages": [("human", "Langsmith에 관해 알려주세요.")]}
)
response["messages"][-1].content

'Langchain과 Langsmith는 관련이 있지만 서로 다른 도구입니다.  Langchain은 LLM 애플리케이션을 구축하기 위한 프레임워크이고, Langsmith는 Langchain 애플리케이션을 위한 추적 및 디버깅 플랫폼입니다.  제공된 API는 Langchain에 대한 정보만 검색할 수 있으므로 Langsmith에 대한 자세한 정보는 제공할 수 없습니다.  Langsmith에 대한 정보를 얻으려면 Langsmith의 공식 웹사이트나 문서를 참조하시기 바랍니다.\n'

### 메시지 스트리밍

In [20]:
# 메시지 스트리밍
for chunk in agent_executor.stream(
    {"messages": [("human", "Langsmith에 관해 알려주세요.")]}
):
    print(chunk)
    print("--")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'langchain_search', 'arguments': '{"query": "Langsmith"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-a8db4884-fcfe-4b1b-8e25-e8e14c3488bd-0', tool_calls=[{'name': 'langchain_search', 'args': {'query': 'Langsmith'}, 'id': '0934599c-ef8f-4239-bfc9-5bc6ea2967ee', 'type': 'tool_call'}], usage_metadata={'input_tokens': 135, 'output_tokens': 7, 'total_tokens': 142, 'input_token_details': {'cache_read': 0}})]}}
--
{'tools': {'messages': [ToolMessage(content='(self-constructing) chainText embedding modelsHow to combine results from multiple retrieversHow to select examples from a LangSmith datasetHow to select examples by lengthHow to select examples by maximal marginal relevance (MMR)How to select examples by n-gram overlapHow to select examples by similarityHow to use reference examples when doing extractio

### 대화 이력을 포함한 에이전트 구현

In [21]:
!pip install langgraph-checkpoint-sqlite

In [61]:
from langgraph.checkpoint.sqlite import SqliteSaver

with SqliteSaver.from_conn_string(":memory:") as memory:

  # 대화 이력을 포함한 에이전트 준비
  agent_executor = chat_agent_executor.create_tool_calling_executor(
      llm,
      tools=[tavily_tool, retriever_tool],
      checkpointer=memory
  )

  # 설정 준비
  config = {"configurable": {"thread_id": "abc123"}}

  # 대화
  for chunk in agent_executor.stream(
      {"messages": [("human", "안녕하세요! 제 이름은 승민입니다.")]},
      config=config
  ):
      print(chunk)
      print("--")

  # 과거 대화 내용에 관해 질문
  for chunk in agent_executor.stream(
      {"messages": [("human", "내 이름은?")]},
      config=config
  ):
      print(chunk)
      print("--")

{'agent': {'messages': [AIMessage(content='안녕하세요, 승민님! 만나서 반가워요. 무엇을 도와드릴까요?\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-e3c2a5ad-0f85-4126-8168-47da97bd0a92-0', usage_metadata={'input_tokens': 153, 'output_tokens': 27, 'total_tokens': 180, 'input_token_details': {'cache_read': 0}})]}}
--
{'agent': {'messages': [AIMessage(content='승민님입니다.\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-85a6416c-c475-42fa-8729-df3715a499b7-0', usage_metadata={'input_tokens': 186, 'output_tokens': 6, 'total_tokens': 192, 'input_token_details': {'cache_read': 0}})]}}
--


In [62]:
with SqliteSaver.from_conn_string(":memory:") as memory:

  # 대화 이력을 포함한 에이전트 준비
  agent_executor = chat_agent_executor.create_tool_calling_executor(
      llm,
      tools=[tavily_tool, retriever_tool],
      checkpointer=memory
  )

  # 설정 준비
  config = {"configurable": {"thread_id": "abc123"}}

  # 검색 도구를 사용한 질의 응답
  for chunk in agent_executor.stream(
      {"messages": [("human", "Langsmith에 관해 알려주세요.")]},
      config=config
  ):
      print(chunk)
      print("--")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'langchain_search', 'arguments': '{"query": "Langsmith"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-6ad2dc99-ae44-43ff-9154-3e84fe01fd9b-0', tool_calls=[{'name': 'langchain_search', 'args': {'query': 'Langsmith'}, 'id': 'caef66a0-89d0-4432-b279-7f8364d3fdbe', 'type': 'tool_call'}], usage_metadata={'input_tokens': 152, 'output_tokens': 7, 'total_tokens': 159, 'input_token_details': {'cache_read': 0}})]}}
--
{'tools': {'messages': [ToolMessage(content='(self-constructing) chainText embedding modelsHow to combine results from multiple retrieversHow to select examples from a LangSmith datasetHow to select examples by lengthHow to select examples by maximal marginal relevance (MMR)How to select examples by n-gram overlapHow to select examples by similarityHow to use reference examples when doing extractio